In [16]:
from anomstack.config import get_specs
from dashboard.data import get_data

In [17]:
specs = get_specs(metrics_dir='../metrics')
spec = specs['weather']
spec


{'db': 'duckdb',
 'table_key': 'metrics_weather',
 'disable_dashboard': False,
 'model_path': 'local://./models',
 'model_configs': [{'model_name': 'PCA',
   'model_tag': 'pca_default',
   'model_params': {'contamination': 0.01}},
  {'model_name': 'KNN',
   'model_tag': 'knn_default',
   'model_params': {'contamination': 0.01}}],
 'model_combination_method': 'mean',
 'train_max_n': 2500,
 'train_metric_timestamp_max_days_ago': 30,
 'train_min_n': 14,
 'score_max_n': 25,
 'score_metric_timestamp_max_days_ago': 7,
 'preprocess_params': {'diff_n': 1,
  'smooth_n': 3,
  'lags_n': 5,
  'freq': None,
  'freq_agg': 'mean'},
 'alert_methods': 'email,slack',
 'alert_always': False,
 'alert_metric_timestamp_max_days_ago': 45,
 'alert_recent_n': 1,
 'alert_max_n': 250,
 'alert_smooth_n': 3,
 'alert_snooze_n': 3,
 'alert_threshold': 0.8,
 'change_metric_timestamp_max_days_ago': 45,
 'change_max_n': 250,
 'change_smooth_n': 1,
 'change_snooze_n': 3,
 'change_threshold': 3.5,
 'change_detect_last_n'

In [18]:
df = get_data(spec)
df.head()

,metric_timestamp,metric_batch,metric_name,metric_value,metric_score,metric_alert,metric_llmalert,metric_change,metadata
0,2025-03-08 14:45:36.486960+00:00,weather,temp_athens,18.0,NaN,NaN,NaN,NaN,[None]
1,2025-03-08 14:45:36.486960+00:00,weather,temp_berlin,17.4,NaN,NaN,NaN,NaN,[None]
2,2025-03-08 14:45:36.486960+00:00,weather,temp_dublin,12.1,NaN,NaN,NaN,NaN,[None]
3,2025-03-08 14:45:36.486960+00:00,weather,temp_london,16.4,NaN,NaN,NaN,NaN,[None]
4,2025-03-08 14:45:36.486960+00:00,weather,temp_paris,19.0,NaN,NaN,NaN,NaN,[None]


In [22]:
df.query("metric_llmalert == 1 & metric_name == 'temp_sydney_uv_index'")


,metric_timestamp,metric_batch,metric_name,metric_value,metric_score,metric_alert,metric_llmalert,metric_change,metadata
102,2025-03-17 21:20:00+00:00,weather,temp_sydney_uv_index,NaN,NaN,NaN,1.0,NaN,[None]
1030,2025-03-26 21:48:06+00:00,weather,temp_sydney_uv_index,NaN,NaN,NaN,1.0,NaN,[None]
